<a href="https://colab.research.google.com/github/pglez82/IFCB_semisupervised/blob/master/IFCB_FT_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# check that we have the required libraries

In this case we are using our IFCB_HDF5 to load the dataset in this format see: https://github.com/pglez82/IFCB_HDF5

In [1]:
import os,sys

if not os.path.isdir("/media/nas/pgonzalez/IFCB_HDF5"):
    print("You should have the IFCB_HDF5 project in this directory to run this notebook")
    raise StopExecution
sys.path.insert(1, os.path.abspath("/media/nas/pgonzalez/IFCB_HDF5"))

In [2]:
import pandas as pd

if not os.path.isfile('IFCB.csv.zip'):
    print("CSV data do not exist. Downloading...")
    !wget -O IFCB.csv.zip "https://unioviedo-my.sharepoint.com/:u:/g/personal/gonzalezgpablo_uniovi_es/EfsVLhFsYJpPjO0KZlpWUq0BU6LaqJ989Re4XzatS9aG4Q?download=1"

data = pd.read_csv('IFCB.csv.zip',compression='infer', header=0,sep=',',quotechar='"')
#Compute sample and year information
data['year'] = data['Sample'].str[6:10].astype(str) #Compute the year
#data["day"] = data["Sample"].str[11:14].astype(str)  # Compute the day
samples=data.groupby('Sample').first()
#samples = samples[["year", "day"]]
samples = samples[["year"]]
print(data)

# Filter by capture device
#samples = samples[samples.index.str.contains("IFCB1")]
#samples.sort_values(["year", "day"], ascending=[True, True])  # Sort by year and day

                        Sample  roi_number        OriginalClass  \
0        IFCB1_2006_158_000036           1                  mix   
1        IFCB1_2006_158_000036           2  Tontonia_gracillima   
2        IFCB1_2006_158_000036           3                  mix   
3        IFCB1_2006_158_000036           4                  mix   
4        IFCB1_2006_158_000036           5                  mix   
...                        ...         ...                  ...   
3457814  IFCB5_2014_353_205141        6850       Leptocylindrus   
3457815  IFCB5_2014_353_205141        6852                  mix   
3457816  IFCB5_2014_353_205141        6855                  mix   
3457817  IFCB5_2014_353_205141        6856                  mix   
3457818  IFCB5_2014_353_205141        6857                  mix   

              AutoClass FunctionalGroup  year  
0                   mix      Flagellate  2006  
1           ciliate_mix         Ciliate  2006  
2                   mix      Flagellate  2006  
3  

In [3]:
import numpy as np
np.unique(data["AutoClass"],return_counts=True)

(array(['Asterionellopsis', 'Cerataulina', 'Ceratium', 'Chaetoceros',
        'Corethron', 'Coscinodiscus', 'Cylindrotheca', 'DactFragCerataul',
        'Dactyliosolen', 'Dictyocha', 'Dinobryon', 'Dinophysis', 'Ditylum',
        'Ephemera', 'Eucampia', 'Euglena', 'Gonyaulax', 'Guinardia',
        'Guinardia_flaccida', 'Guinardia_striata', 'Gyrodinium',
        'Hemiaulus', 'Laboea', 'Lauderia', 'Leptocylindrus', 'Licmophora',
        'Myrionecta', 'Odontella', 'Paralia', 'Phaeocystis', 'Pleurosigma',
        'Prorocentrum', 'Pseudonitzschia', 'Pyramimonas', 'Rhizosolenia',
        'Skeletonema', 'Stephanopyxis', 'Thalassionema', 'Thalassiosira',
        'Thalassiosira_dirty', 'bad', 'ciliate_mix', 'clusterflagellate',
        'detritus', 'dino30', 'kiteflagellates', 'mix', 'mix_elongated',
        'na', 'pennate', 'tintinnid'], dtype=object),
 array([   1729,   21556,     328,   40965,    3028,     279,   19516,
           5877,    4918,    1074,    3510,     154,    1352,     261,
   

# Create training set

Here we make a reestructuration of the images depending on which years we consider

In [3]:
from tqdm import tqdm
import numpy as np
import random

random.seed(2032)

tqdm.pandas()

classcolumn = "AutoClass" #AutoClass means 51 classes
yearstrainingval = ['2006','2007','2008'] #Years to consider as training and validation
yearstest = ['2009','2010','2011','2012','2013','2014'] #Years to consider as test

samplestrainingval = list(samples[samples['year'].isin(yearstrainingval)].index)
random.shuffle(samplestrainingval) 
split_index = int(len(samplestrainingval) * 0.7)
samplestraining = samplestrainingval[:split_index]
samplesval = samplestrainingval[split_index:]

samplestest = list(samples[samples['year'].isin(yearstest)].index) #Samples to consider for testing

classes=np.unique(data[classcolumn])
classes.sort()
classes = np.delete(classes, 21) #This class do not exist in the training set, if we find it in validation we will consider it as mix

# Configure the process

In [4]:
import torch,torchvision
import random
import numpy as np

print("Using pytorch {}".format(torch.__version__))

torch.manual_seed(0) #Reproducible
random.seed(0) #it seems that the transforms uses this random
np.random.seed(0)

gpus = [0,1] #gpus to use
num_gpus = len(gpus)

num_workers = 6*num_gpus
batch_size = 512 #512 for resnet 18, and 34 (with 2 gpus). 256 for resnet 50
batch_size_val = 512

print("Num workers {}. Batch size training {}. Batch size validation {}".format(num_workers,batch_size,batch_size_val))

num_epochs_ft1 = 10 # @param
num_epochs_ft2 = 10 # @param

base_model = torchvision.models.resnet34(pretrained=True) #From which model to start

is_trained=False #Will take true if the network is already trained
model_save_path="models/model34_64px_byyears_zenodo.pt" #Where to save the model once trained
results_save_path="results_model34_64px_byyears_zenodo" #Where to save all the predictions for the training and testing samples

hdf5_files_path = '/media/nas/pgonzalez/IFCB_HDF5/output/' #Directory with the dataset

if not os.path.isdir(results_save_path):
    os.mkdir(results_save_path)
    os.mkdir(os.path.join(results_save_path,'train'))
    os.mkdir(os.path.join(results_save_path,'test'))
else:
    print("Output directory already exists, will override everything there...")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using %s"%device)

Using pytorch 1.10.0
Num workers 12. Batch size training 512. Batch size validation 512
Output directory already exists, will override everything there...
Using cuda:0


# Prepare de DataLoaders for the CNN
In this step it is important to consider that we have to use images with the same size than the original network (so we can reuse the weights)

In [6]:
import torchvision.transforms as T
from ifcb_h5py.h5ifcbdataset import H5IFCBDataset
from torch.utils.data import DataLoader


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

#Define transofrmations
train_transform = T.Compose([
  T.Resize(size=64),
  T.RandomResizedCrop(size=64),
  T.RandomHorizontalFlip(),
  T.ToTensor(),            
  #T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

val_transform = T.Compose([
  T.Resize(size=64),
  T.CenterCrop(size=64),
  T.ToTensor(),
  #T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

#files to load
filestraining = [hdf5_files_path+s+'.hdf5' for s in samplestraining]
filesval = [hdf5_files_path+s+'.hdf5' for s in samplesval]

#Define data loaders
train_dset = H5IFCBDataset(filestraining,classes,classattribute=classcolumn, verbose=1,trainingset=True,transform=train_transform)
train_loader = DataLoader(train_dset,batch_size=batch_size,num_workers=num_workers,shuffle=True,pin_memory=True)

val_dset = H5IFCBDataset(filesval,classes,classattribute=classcolumn, verbose=1,trainingset=False,transform=val_transform)
val_loader = DataLoader(val_dset,batch_size=batch_size,num_workers=num_workers,shuffle=False,pin_memory=True)

Loading samples: 100%|██████████| 86/86 [02:10<00:00,  1.52s/it]


# Define how to load the CNN
In this step we download a pretrained CNN with the weights from ImageNet. We change the last layer to match the number of classes that we have in our problem. In the case that model_trained_path is true, that means that we have already trained the network so we load the weights.

In [7]:
import torch.nn as nn

def load_network():
    global is_trained
    global base_model
    
    model = base_model
    print("Adjusting the CNN for %s classes" % len(classes))
    model.fc = nn.Linear(model.fc.in_features, len(classes))

    print("Let's use", len(gpus), "GPUs!")
    model = nn.DataParallel(model,device_ids=gpus)

    #Define loss function
    loss_fn = nn.CrossEntropyLoss()
    if os.path.isfile(model_save_path):
        model.load_state_dict(torch.load(model_save_path))
        is_trained=True
    
    model = model.to(device) #Send model to gpu
    return model,loss_fn

# Define finetuning util functions

In [8]:
import time
import torch.nn.functional as nnf


def run_epoch(model, loss_fn, loader, optimizer, device):
    """
    Train the model for one epoch.
    """
    loss_epoch = 0 
    start_time = time.time()
    # Set the model to training mode
    model.train()
    for step, (x, y, _) in enumerate(loader):
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)

        # Run the model forward to compute scores and loss.
        scores = model(x)
        loss = loss_fn(scores, y)
        loss_epoch = loss_epoch + loss.item()
        
        # Run the model backward and take a step using the optimizer.
        loss.backward()
        optimizer.step()

        if step % 50== 0 and step!=0:
            spent = time.time()-start_time
            print(f"Step [{step}/{len(loader)}]\t Loss: {loss_epoch/step} \t Time: {spent} secs [{(batch_size*50)/spent} ej/sec]]")
            start_time = time.time()

    return loss_epoch

#Function for a validation epoch
def run_epoch_val(model, loader, device):
    with torch.no_grad():
        loss_epoch = 0 
        # Set the model to evaluation mode
        model.eval()
        for x, y, _ in loader:
            x = x.to(device)
            y = y.to(device)

            # Run the model forward to compute scores and loss.
            scores = model(x)
            loss = loss_fn(scores, y)
            loss_epoch = loss_epoch + loss.item()
            #Compute accuracy
            _,preds = torch.max(scores,1)
            corrects = torch.sum(preds == y)
    return corrects.item()/len(y)

def make_preds(model, loader, device):
    """
    Check the accuracy of the model.
    """
    modeldeepfeatures =  nn.Sequential(*list(model.module.children())[:-1])
    with torch.no_grad():
        # Set the model to eval mode
        model.eval()
        y_true = []
        y_pred = []
        y_logits = []
        deepfeatures = []
        y_probs = []
        sample = []
        for x, y, s in loader: #The idea is that the dataloader can give me the sample of the image so we can return it
            x = x.to(device)
            y = y.to(device)
            # Run the model forward, and compare the argmax score with the ground-truth
            # category.
            output = model(x)
            predicted = output.argmax(1)
            prob = nnf.softmax(output, dim=1)
            y_logits.extend(output.cpu().detach().numpy())
            df = modeldeepfeatures(x).view(-1,512).cpu().detach().numpy()
            deepfeatures.extend(df)
            y_probs.extend(prob.cpu().detach().numpy())
            y_true.extend(y.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            sample.extend(s)
    return y_true,y_pred,y_probs,y_logits,deepfeatures,sample

# Define the finetuning
First we only update the last layer for a few epochs, then we update all the weights with a small learning rate

In [9]:
def finetune(model,loss_fn,train_loader,device):
    best_val_acc=0
    for param in model.parameters():
        param.requires_grad = False
    for param in model.module.fc.parameters():
        param.requires_grad = True

    optimizer = torch.optim.Adam(model.module.fc.parameters(), lr=1e-3)

    #First phase of finetuning
    for epoch in range(num_epochs_ft1):
        # Run an epoch over the training data.
        print('Starting epoch %d / %d' % (epoch + 1,num_epochs_ft1))
        loss_epoch = run_epoch(model, loss_fn, train_loader, optimizer, device)

        # Check accuracy on the train and val sets.
        print(f"Epoch [{epoch+1}/{num_epochs_ft1}]\t Loss: {loss_epoch / len(train_loader)}")

        print("Performing validation...")
        val_accuracy = run_epoch_val(model, val_loader, device)
        print("Validation accuracy %.5f" % val_accuracy)
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(model.state_dict(), model_save_path)
            print("Saved best model in this epoch based on val accuracy")

    #Allow updating all the weights in the second phase
    for param in model.parameters():
        param.requires_grad = True

    #Lower learning rate this time
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

    # Train the entire model for a few more epochs, checking accuracy on the
    # train sets after each epoch.
    for epoch in range(num_epochs_ft2):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs_ft2))
        loss_epoch = run_epoch(model, loss_fn, train_loader, optimizer, device)

        print(f"Epoch [{epoch+1}/{num_epochs_ft2}]\t Loss: {loss_epoch / len(train_loader)}")

        print("Performing validation...")
        val_accuracy = run_epoch_val(model, val_loader, device)
        print("Validation accuracy %.5f" % val_accuracy)
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(model.state_dict(), model_save_path)
            print("Saved best model in this epoch based on val accuracy")
    
    print("Fine tune done, reloading best model.")
    model.load_state_dict(torch.load(model_save_path))
 


# Predict all validations samples

The idea here is to compute the predictions for all the validation samples. We have finetuned the network to the plankton problem and now we iterate over the test samples and compute the output probabilties (file _probs.csv). We also store the _pred (for quantifiers that do not need probability) and the _true label (for checking errors). This is done for each sample so three files will be written per test sample.

In [10]:
from sklearn.metrics import accuracy_score

print('Starting process...')

model,loss_fn = load_network()

if not is_trained:
    finetune(model,loss_fn,train_loader,device)
else:
    print("Model was trained already")

print("Computing deep features for test images...")
path = os.path.join(results_save_path,'test')
for index,sample in enumerate(samplestest):
    file = [hdf5_files_path+sample+'.hdf5']
    test_dset = H5IFCBDataset(file,classes,classattribute=classcolumn,verbose=0,trainingset=False,defaultclass='mix',transform=val_transform)
    test_loader = DataLoader(test_dset,batch_size=batch_size_val,num_workers=num_workers)
    y_true,y_pred,y_probs,y_logits,deepfeatures,_ = make_preds(model, test_loader, device)
    print("Sample {} ({}/{}) -> Accuracy: {}".format(sample,index+1,len(samplestest),accuracy_score(y_true,y_pred)))
    np.savetxt("{}/{}_true.csv".format(path,sample),y_true,fmt='%d')
    np.savetxt("{}/{}_pred.csv".format(path,sample),y_pred,fmt='%d')
    np.savetxt("{}/{}_probs.csv".format(path,sample), y_probs, delimiter=",",fmt='%1.8f')
    np.savetxt("{}/{}_logits.csv".format(path,sample), y_logits, delimiter=",",fmt='%f')
    np.savetxt("{}/{}_deepfeatures.csv".format(path,sample), deepfeatures, delimiter=",",fmt='%f')


print("Predicting also train and val samples...")
path = os.path.join(results_save_path,'train')
for index,sample in enumerate(samplestrainingval):
    file = [hdf5_files_path+sample+'.hdf5']
    train_dset = H5IFCBDataset(file,classes,classattribute=classcolumn,verbose=0,trainingset=False,defaultclass='mix',transform=val_transform)
    train_loader = DataLoader(train_dset,batch_size=batch_size_val,num_workers=num_workers)
    y_true,y_pred,y_probs,y_logits,deepfeatures,_ = make_preds(model, train_loader, device)
    print("Sample {} ({}/{}) -> Accuracy: {}".format(sample,index+1,len(samplestrainingval),accuracy_score(y_true,y_pred)))
    np.savetxt("{}/{}_true.csv".format(path,sample),y_true,fmt='%d')
    np.savetxt("{}/{}_pred.csv".format(path,sample),y_pred,fmt='%d')
    np.savetxt("{}/{}_probs.csv".format(path,sample), y_probs, delimiter=",",fmt='%1.8f')
    np.savetxt("{}/{}_logits.csv".format(path,sample), y_logits, delimiter=",",fmt='%f')
    np.savetxt("{}/{}_deepfeatures.csv".format(path,sample), deepfeatures, delimiter=",",fmt='%f')

Starting process...
Adjusting the CNN for 50 classes
Let's use 2 GPUs!
Starting epoch 1 / 10
Step [50/1141]	 Loss: 1.4588942539691925 	 Time: 6.756144285202026 secs [3789.143469903632 ej/sec]]
Step [100/1141]	 Loss: 1.1892353880405426 	 Time: 2.404506206512451 secs [10646.676615208577 ej/sec]]
Step [150/1141]	 Loss: 1.0715463121732076 	 Time: 2.3832671642303467 secs [10741.556961897419 ej/sec]]
Step [200/1141]	 Loss: 0.9999886125326156 	 Time: 2.3159477710723877 secs [11053.78986510825 ej/sec]]
Step [250/1141]	 Loss: 0.9497596781253814 	 Time: 2.397792339324951 secs [10676.487525690882 ej/sec]]
Step [300/1141]	 Loss: 0.9149046305815379 	 Time: 2.5111584663391113 secs [10194.498014822984 ej/sec]]
Step [350/1141]	 Loss: 0.8875687289237976 	 Time: 2.343639612197876 secs [10923.181135341965 ej/sec]]
Step [400/1141]	 Loss: 0.8665475137531757 	 Time: 2.4108335971832275 secs [10618.73371513926 ej/sec]]
Step [450/1141]	 Loss: 0.848936259481642 	 Time: 2.5936858654022217 secs [9870.123572590015

### Compute the prevalences for the test samples

In [11]:
path = os.path.join(results_save_path,'test')
true_prevalences = pd.DataFrame(columns=classes,index=samplestest.sort(),dtype=float)
for index,sample in enumerate(samplestest):
    y_true = np.loadtxt("{}/{}_true.csv".format(path,sample),dtype=int)
    for i,cl in enumerate(classes):
        true_prevalences.loc[sample,cl]=sum(y_true==i)
    true_prevalences.loc[sample]=true_prevalences.loc[sample]/sum(true_prevalences.loc[sample])
true_prevalences.to_csv(os.path.join(results_save_path,'test_prevalences.csv'),float_format='%.5f',index_label='sample')

### Format train files
Each sample would be a file, with the label and then the features


In [14]:
from tqdm import tqdm

path = os.path.join(results_save_path, 'train')
column_names = np.append('class', np.arange(512).astype(str))

for sample in tqdm(samplestrainingval):
    y = np.loadtxt("{}/{}_true.csv".format(path, sample), dtype=int)
    
    deepfeatures = np.loadtxt("{}/{}_deepfeatures.csv".format(path, sample), delimiter=',')

    assert y.shape[0] == deepfeatures.shape[0], "Should have the same dimension"
    assert deepfeatures.shape[1] == 512, "Should have 512 features"

    # Create a single DataFrame for the entire sample
    df = pd.DataFrame(np.hstack((classes[y.reshape(-1, 1)], deepfeatures)), columns=column_names)

    # Save the DataFrame to a CSV file
    df['class'] = df['class'].astype(str)
    df.to_csv(os.path.join(path, f"{sample}.csv"), float_format='%.5f', index=False)
    

100%|██████████| 286/286 [09:14<00:00,  1.94s/it]


### Format test files


In [13]:
from tqdm import tqdm

path = os.path.join(results_save_path, 'test')
column_names = np.arange(512).astype(str)

for sample in tqdm(samplestest):
    deepfeatures = np.loadtxt("{}/{}_deepfeatures.csv".format(path, sample), delimiter=',')

    assert deepfeatures.shape[1] == 512, "Should have 512 features"

    # Create a single DataFrame for the entire sample
    df = pd.DataFrame(deepfeatures, columns=column_names)

    # Save the DataFrame to a CSV file
    df.to_csv(os.path.join(path, f"{sample}.csv"), float_format='%.5f', index=False)

100%|██████████| 678/678 [31:22<00:00,  2.78s/it]


In [16]:
import zipfile

os.mkdir('export')

#test prevalences
zip_file = zipfile.ZipFile('export/IFCB.test_prevalences.zip', mode='w')
file_to_add = os.path.join(results_save_path, "test_prevalences.csv")
filename = os.path.basename(file_to_add)
zip_file.write(file_to_add, arcname=filename)
zip_file.close()

paths = [os.path.join(results_save_path,'train',s+'.csv') for s in samplestrainingval]
print("Exporting train samples:")
with zipfile.ZipFile(os.path.join('export', 'IFCB.train.zip'), 'w') as zip_file:
    for sample in tqdm(samplestrainingval):
        file_path = os.path.join(results_save_path, 'train', sample + '.csv')
        arcname = os.path.join('train',os.path.basename(file_path))
        zip_file.write(file_path, arcname=arcname, compress_type=zipfile.ZIP_DEFLATED)
zip_file.close()

print("Exporting test samples:")
with zipfile.ZipFile(os.path.join('export', 'IFCB.test.zip'), 'w') as zip_file:
    for sample in tqdm(samplestest):
        file_path = os.path.join(results_save_path, 'test', sample + '.csv')
        arcname = os.path.join('test',os.path.basename(file_path))
        zip_file.write(file_path, arcname=arcname, compress_type=zipfile.ZIP_DEFLATED)
zip_file.close()


Exporting train samples:


  0%|          | 0/286 [00:00<?, ?it/s]

100%|██████████| 286/286 [03:12<00:00,  1.48it/s]


Exporting test samples:


100%|██████████| 678/678 [09:27<00:00,  1.19it/s]
